In [28]:
import skmob
import pandas as pd
import csv, os
from skmob.preprocessing import detection, clustering

device_id_list = []
for it in os.scandir('./staypoint'):
    if it.is_dir():
        if it.name.isdigit():
            device_id_list.append(it.name)
device_id_list.sort()

for device_id in device_id_list:
    stdf = skmob.TrajDataFrame.from_file('./staypoint/'+ device_id +'/stay_points.csv',
        latitude='latitude', longitude='longitude', user_id='deviceid', datetime='datetime')

    stdf['leaving_datetime'] = pd.to_datetime(stdf['leaving_datetime'])

    # m = stdf.plot_trajectory(max_users=1, start_end_markers=False)
    # stdf.plot_stops(max_users=1, map_f=m)

    cstdf = clustering.cluster(stdf, cluster_radius_km=0.2, min_samples=1)

    stay_point_list = cstdf.values.tolist()
    cluster_list = list()

    for stay_point in stay_point_list:
        stay_point[3] = stay_point[3].strftime('%Y-%m-%d %H:%M:%S')
        stay_point[4] = stay_point[4].strftime('%Y-%m-%d %H:%M:%S')
        if stay_point[5] not in cluster_list:
            cluster_list.append(stay_point[5])
    cluster_list.sort()
    
    fields = ['deviceid', 'latitude', 'longitude','datetime', 'leaving_datetime', 'cluster']
    path = "./cluster/"+device_id   
    if not os.path.isdir(path):
        os.mkdir(path)
    with open("./cluster/"+device_id+'/clustering_result.csv', 'w', newline='') as f:
        write = csv.writer(f)
        write.writerow(fields)
        write.writerows(stay_point_list)
    
    for c in cluster_list:
        f = open("./cluster/"+device_id+"/cluster_"+str(c)+'.csv', 'w', newline='')
        write = csv.writer(f)
        write.writerow(fields)
        for stay_point in stay_point_list:
            if(stay_point[5]==c):
                write.writerow(stay_point)
        f.close()

In [17]:
def get_week(device_id):
    stdf = skmob.TrajDataFrame.from_file('./staypoint/53/stay_points.csv',
                                         latitude='latitude', longitude='longitude', user_id='deviceid', datetime='arrive_time')

    stdf.rename(columns={'leaving_time': 'leaving_datetime'}, inplace=True)
    stdf['leaving_datetime'] = pd.to_datetime(stdf['leaving_datetime'])

    weeks = [g for n, g in stdf.set_index(
        'datetime').groupby(pd.Grouper(freq='W'))]

    return weeks

a = get_week(53)
print(a)

[                     uid        lat         lng    leaving_datetime
datetime                                                           
2019-10-24 11:37:58   53  37.570761  127.034212 2019-10-24 13:42:10
2019-10-24 14:09:18   53  37.570762  127.034217 2019-10-24 17:39:30
2019-10-24 18:21:04   53  37.560932  127.032245 2019-10-24 19:03:08
2019-10-24 19:18:16   53  37.560945  127.032240 2019-10-25 08:21:12
2019-10-25 08:25:18   53  37.564035  127.036923 2019-10-25 09:41:02
2019-10-25 12:17:50   53  38.076584  127.320265 2019-10-25 13:04:01
2019-10-25 14:23:11   53  38.067250  127.325462 2019-10-25 15:27:01
2019-10-25 17:53:39   53  37.562298  127.035562 2019-10-25 21:01:18
2019-10-25 21:12:30   53  37.560922  127.032230 2019-10-26 17:49:13
2019-10-26 17:53:16   53  37.560868  127.038064 2019-10-26 18:27:34
2019-10-26 18:27:34   53  37.559276  127.034890 2019-10-26 22:04:57
2019-10-26 22:07:01   53  37.560921  127.032232 2019-10-27 13:02:46
2019-10-27 14:08:45   53  37.569242  126.978103